# 01. Setup and Data Exploration

This notebook verifies the environment setup and explores the raw 7T data.

In [ ]:
import os
import sys
sys.path.append('../')

import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from nilearn import plotting

from src.io import find_bids_files, load_nifti
from src.visualization import plot_slice_visualization

print("Libraries imported successfully.")

## 1. Check Data Directory

Ensure the AHEAD data is present in the `data/` directory.

In [ ]:
DATA_DIR = '../data'

subjects = [d for d in os.listdir(DATA_DIR) if d.startswith('sub-')]
print(f"Found {len(subjects)} subjects: {subjects}")

if not subjects:
    print("WARNING: No subjects found. Please download AHEAD data and place it in the data/ directory.")

## 2. Load and Visualize One Subject

We will load the T1w and R2* maps for the first subject.

In [ ]:
if subjects:
    sub_id = subjects[0]
    print(f"Analyzing subject: {sub_id}")
    
    # Find files (adjust suffixes based on actual BIDS naming in AHEAD)
    t1w_files = find_bids_files(DATA_DIR, sub_id, 'T1w.nii.gz')
    r2s_files = find_bids_files(DATA_DIR, sub_id, 'R2star.nii.gz')
    
    if t1w_files:
        t1w_img = load_nifti(t1w_files[0])
        print(f"T1w Shape: {t1w_img.shape}")
        plot_slice_visualization(t1w_img, title=f"{sub_id} T1w")
    else:
        print("T1w file not found.")
        
    if r2s_files:
        r2s_img = load_nifti(r2s_files[0])
        print(f"R2* Shape: {r2s_img.shape}")
        plot_slice_visualization(r2s_img, title=f"{sub_id} R2*", cmap='hot')
    else:
        print("R2* file not found.")